In [1]:

!pip install pyswarms


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 2.5 MB/s eta 0:00:00


In [2]:

import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from pyswarms.single import GlobalBestPSO

In [3]:
# Load dataset
df = pd.read_csv('wearaz.csv')

In [4]:
X = df.drop('Output', axis = 1).values
y = df['Output'].values

In [5]:
# Split dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:

from sklearn.ensemble import GradientBoostingRegressor

In [7]:

# Define the objective function to optimize the Gradient Boosting model using PSO
def objective_function(params):
    n_particles = params.shape[0]
    mse_particles = []

    for i in range(n_particles):
        learning_rate, n_estimators, max_depth, min_samples_split = params[i]

        # Set up the Gradient Boosting model
        model = GradientBoostingRegressor(learning_rate=learning_rate,
                                          n_estimators=int(n_estimators),
                                          max_depth=int(max_depth),
                                          min_samples_split=int(min_samples_split))

        # Train the model
        model.fit(X_train, y_train)

        # Predict the target values
        y_pred = model.predict(X_test)

        # Calculate MSE
        mse = mean_squared_error(y_test, y_pred)
        mse_particles.append(mse)

    return np.array(mse_particles)

In [8]:

# Define the search space for PSO
lower_bounds = np.array([0.01, 100, 2, 2])
upper_bounds = np.array([0.3, 500, 10, 20])
bounds = (lower_bounds, upper_bounds)

# Initialize PSO
options = {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
optimizer = GlobalBestPSO(n_particles=10, dimensions=len(lower_bounds), options=options, bounds=bounds)

# Perform optimization
cost, pos = optimizer.optimize(objective_function, iters=30)

# Train the final Gradient Boosting model with the best parameters found by PSO
best_model = GradientBoostingRegressor(learning_rate=pos[0],
                                       n_estimators=int(pos[1]),
                                       max_depth=int(pos[2]),
                                       min_samples_split=int(pos[3]))

best_model.fit(X_train, y_train)

# Make predictions using the final model
y_pred = best_model.predict(X_test)

# Calculate the evaluation metrics
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error (MSE): {mse}')
print(f'Mean Absolute Error (MAE): {mae}')
print(f'R-squared (R2): {r2}')

2023-06-24 15:55:05,581 - pyswarms.single.global_best - INFO - Optimize for 30 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
pyswarms.single.global_best: 100%|██████████|30/30, best_cost=1.8e-9
2023-06-24 15:57:35,993 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 1.7957790835640959e-09, best pos: [5.72742251e-02 4.16979934e+02 8.27473163e+00 8.97654344e+00]


Mean Squared Error (MSE): 1.8117916359535276e-09
Mean Absolute Error (MAE): 3.17945410106165e-05
R-squared (R2): 0.9997029544289009
